<a href="https://colab.research.google.com/github/josephgills/DataSociety/blob/main/WeatherGrab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temperature Grab (List Input)
In this block of code we define the following functions:
- parse_lat_lon: extracts a latitude and longitude in decimal format
- get_forecast_period_by_name: extracts the forecast from the api response for a given period
- get_forecast_temperature: retrieves the temperature from a forecast for a given period at a specific (lat, lon) position
- get_temperatures: retrieves the temperatures from a forecast for a given period and a list of specific (lat, lon) positions

## To Run:
Hit the Play Button below and enter the desired lat,lon positions seperated by a semi-colon. When prompted enter the desired period.

##Sample Input:
38.2527° N, 85.7585° W; 38,-84

Wednesday Night

In [44]:
import re
import requests

def parse_lat_lon(input_string):
    # Regular expression to extract latitude and longitude
    pattern = r'(\d+\.\d+)°?\s*([NS]),?\s*(\d+\.\d+)°?\s*([EW])'
    match = re.match(pattern, input_string)

    if match:
        # Extract latitude and longitude values
        lat_deg = float(match.group(1))
        lat_dir = 1 if match.group(2) == 'N' else -1
        lon_deg = float(match.group(3))
        lon_dir = 1 if match.group(4) == 'E' else -1

        # Convert degrees to decimal format
        latitude = lat_deg * lat_dir
        longitude = lon_deg * lon_dir

        return latitude, longitude, False
    else:
        try:
            # Split the input string by comma and whitespace
            parts = input_string.split(',')
            latitude = float(parts[0])
            longitude = float(parts[1])
            return latitude, longitude, True
        except Exception as e:
            print("Invalid input format.")
            return None, None, False

def get_forecast_period_by_name(period_name, forecast_data):
    periods = forecast_data['properties']['periods']
    period_mapping = {period['name']: period for period in periods}
    return period_mapping.get(period_name)

def get_forecast_temperature(lat_lon, period_name):
    forecast_url = f"https://api.weather.gov/points/{lat_lon[0]},{lat_lon[1]}"

    # Fetch metadata
    response = requests.get(forecast_url)

    if response.status_code == 200:
        metadata = response.json()

        # Get forecast URL from metadata
        forecast_url = metadata['properties']['forecast']

        # Fetch forecast data
        response = requests.get(forecast_url)

        if response.status_code == 200:
            forecast_data = response.json()

            # Get the forecast period by name
            period = get_forecast_period_by_name(period_name, forecast_data)

            if period:
                return period['temperature'], period['temperatureUnit']
            else:
                return None, None
        else:
            print(f"Failed to fetch forecast data: {response.status_code}")
            return None, None
    else:
        print(f"Failed to fetch metadata: {response.status_code}")
        return None, None

def get_temperatures(input_string, period_name):
    # Split input by semi-colons
    lat_lon_list = input_string.split(';')

    for lat_lon_str in lat_lon_list:
        latitude, longitude, is_decimal = parse_lat_lon(lat_lon_str.strip())

        if latitude is not None and longitude is not None:
            if is_decimal:
                print(f"Input: {latitude}, {longitude}")
            else:
                print(f"Input: {latitude}° N, {longitude}° W")

            temperature, unit = get_forecast_temperature((latitude, longitude), period_name)
            if temperature and unit:
                print(f"Temperature for {period_name}: {temperature}°{unit}")
            else:
                print(f"Temperature for {period_name} not available.")
            print()
        else:
            print("Invalid input format.")

if __name__ == "__main__":
    # User input
    input_string = input("Enter a list of latitude and longitude pairs (in decimal format) or in the format '38.2527° N, 85.7585° W', separated by semi-colons: ")
    period_name = input("Select the forecast period name (e.g., 'Wednesday', 'Wednesday Night', etc.): ")
    get_temperatures(input_string,period_name);

Enter a list of latitude and longitude pairs (in decimal format) or in the format '38.2527° N, 85.7585° W', separated by semi-colons: 38.2527° N, 85.7585° W; 38,-84
Select the forecast period name (e.g., 'Wednesday', 'Wednesday Night', etc.): Wednesday Night
Input: 38.2527° N, -85.7585° W
Temperature for Wednesday Night: 29°F

Input: 38.0, -84.0
Temperature for Wednesday Night: 26°F

